### 관광지 대중소분류와 MBTI 키워드 추출 데이터 유사도 계산 및 데이터 분할
### 맛집 리뷰 키워드 추출 데이터와, MBTI 키워드 추출 데이터 유사도 계산 및 데이터 분할

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

/opt/homebrew/Caskroom/miniforge/base/envs/bigcon/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import importlib.metadata

def check_package_version(package_name):
    try:
        version = importlib.metadata.version(package_name)
        print(f"{package_name} 버전: {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"{package_name}이(가) 설치되지 않았습니다.")

# 확인하려는 패키지 이름
check_package_version("libfaiss")

libfaiss이(가) 설치되지 않았습니다.


In [2]:
with open('tour_type/tour_type/MBTI별_맛집_음식성향_키워드.json', 'r', encoding='utf-8') as file:
    mbti = json.load(file)

- mbti 키워드 전처리 및 임베딩

In [3]:
mbti_df = pd.DataFrame(columns=['mbti', 'keyword'], data=mbti.items())

In [4]:
mbti_df['keyword'] = mbti_df['keyword'].apply(lambda x: x[x.find(':')+1:])
mbti_df['keyword'] = mbti_df['keyword'].apply(lambda x: x.replace('\n', ''))
mbti_df['keyword'] = mbti_df['keyword'].apply(lambda x: x.replace('#', ''))
mbti_df['keyword'] = mbti_df['keyword'].apply(lambda x: x.replace('*', ''))
#mbti_df

,mbti,keyword
0,INTJ,"숨겨진 보석, 덜 알려진 곳, 역사, 문화, 자연, 숨겨진 해변, 고즈넉한 오름,..."
1,INTP,"분석적 탐험, 독립적인 여행, 숨겨진 매력, 자신만의 이야기, 인적 드문 자연..."
2,ENTJ,"숨겨진 명소, 덜 알려진 곳, 역사, 문화, 탐험, 독특한 숙소, 편안함, 고급스..."
3,ENTP,"숨겨진 보석, 숨겨진 명소, 현지인 추천, 덜 알려진 곳, 역사, 문화, 예술, ..."
4,INFJ,"에코 투어, 지속 가능한 여행, 자연 친화적, 의미 있는 경험, 숨겨진 명소, 아..."
5,INFP,"자연, 힐링, 감성, 휴식, 캠핑, 혼자 여행, 특별한 의미, 스토리, 예술, 아름..."
6,ENFJ,"따뜻함, 의미, 휴양, 체험, 감성, 풍경, 골목길, 예술, 소통, 교류, 역..."
7,ENFP,"제주도, ENFP, 여행, 맛집, 로드트립, 숨겨진 비경, 문화, 현지인, 소통, ..."
8,ISTJ,"꼼꼼한 계획, 익숙함, 편안함, 후기, 평점, 전통적인 분위기, 유명 관광지, 혼자..."
9,ISFJ,"익숙한 맛, 편안함, 꼼꼼한 계획, 안전, 편리, 유명 관광 명소, 후기, 인기 ..."


- 리뷰 키워드 임베딩

In [5]:
review_ms = pd.read_csv('review_keyword/review_keywords_민성.csv')
review_yj = pd.read_csv('review_keyword/review_keywords_윤지.csv')
review_hb = pd.read_csv('review_keyword/review_keywords_형빈.csv')

In [6]:
review_ms.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
review = pd.concat([review_ms, review_yj, review_hb])

In [10]:
review.reset_index(drop=True, inplace=True)

In [11]:
# 임베딩 모델

'''
[ List ]
- intfloat/multilingual-e5-large-instruct
- jhgan/ko-sroberta-multitask
- upskyy/bge-m3-Korean
'''
# model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')
model = SentenceTransformer('upskyy/bge-m3-Korean')
# model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [12]:
def get_embedding(text):
    return model.encode(text)

# def get_average_embedding_mbti(keywords):
#     embeddings = [get_embedding(keyword) for keyword in keywords]
#     return np.mean(embeddings, axis=0)

# def get_average_embedding_review(keywords):
#     keywords = keywords.strip()[1:-1].strip()
#     embeddings = [get_embedding(keyword) for keyword in keywords.split(',')]
#     return np.mean(embeddings, axis=0)

def calculate_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

In [13]:
# 임베딩 값 계산해 컬럼으로 추가
mbti_df['embedding'] = mbti_df['keyword'].apply(lambda x: get_embedding(str(x)))

In [15]:
review['tfidf_embedding'] = review['tf_idf_keywords'].apply(lambda x: get_embedding(str(x).replace('[', '').replace(']', '').replace("'",'')))

In [16]:
review['wordrank_embedding'] = review['krword_rank'].apply(lambda x: get_embedding(str(x).replace('[', '').replace(']', '').replace("'",'')))

In [17]:
review['llm_embedding'] = review['LLM_keywords'].apply(lambda x: get_embedding(str(x).replace('[', '').replace(']', '').replace("'",'')))

- tf-idf, kr word rank 임베딩 값과 llm 임베딩 값 가중치 평균

In [18]:
review['llm5_other5'] = 0.25*(review['tfidf_embedding']) + 0.25*(review['wordrank_embedding']) + 0.5*(review['llm_embedding']) #; review

In [19]:
review.to_csv('mbti_review_similarity_llm5_other5.csv', index=False)

### 관광지와 MBTI 유형 간 유사도 상위 N개 저장

In [33]:
# 관광지 월별 데이터 불러오기

tour_dfs = {}
import os
from glob import glob
csvs = glob('./data/tour_df/original/*.csv')
csvs.sort()
for i, j in enumerate(csvs):
    tour_dfs[f"tour_{i+1}"] = pd.read_csv(j)


- 관광지 데이터 임베딩

In [41]:
tour_df = tour_dfs['tour_1'].iloc[:,[0,2,3,4,5]]
tour_df['키워드'] = tour_df['장소 카테고리'] + ', ' + tour_df['대분류'] + ', ' + tour_df['중분류'] + ', ' + tour_df['소분류']
tour_df = tour_df[tour_df['장소 카테고리'].apply(lambda x: x != "음식점")]
tour_df['embedding'] = tour_df['키워드'].apply(lambda x: get_embedding(x))
#tour_df

,관광지명,장소 카테고리,대분류,중분류,소분류
0,가름게스트하우스,숙박,숙박,숙박시설,펜션
1,가마오름,관광지,자연,자연관광지,산
2,가문이오름,관광지,자연,자연관광지,산
3,가세오름,관광지,자연,자연관광지,산
4,가시식당,음식점,음식,음식점,한식
...,...,...,...,...,...
821,5.16도로숲터널,관광지,인문(문화/예술/역사),건축/조형물,터널
822,513텐동,음식점,음식,음식점,일식
823,73st,음식점,음식,음식점,카페/전통찻집
824,9.81파크,레포츠,레포츠,육상 레포츠,카트


In [84]:
tour_df_emb = pd.DataFrame()

In [85]:
from tqdm import tqdm
# 각 MBTI 유형에 대해 llm 0.5 other 0.5 유사도 계산
num = 0
for place, place_embedding in tqdm(tour_df[['관광지명','embedding']].values, desc='place'):
    for mbti, mbti_embedding in mbti_df[['mbti','embedding']].values:
    
        similarity = calculate_similarity(place_embedding, mbti_embedding)
        #print(f"{mbti} 유형과 {name} 키워드의 유사도: {similarity:.6f}")
        tour_df_emb.loc[num, '관광지명'] = place
        tour_df_emb.loc[num, mbti] = similarity
    num += 1

place: 100%|██████████| 478/478 [00:02<00:00, 222.18it/s]


In [86]:
tour_df_mbti = {} # mbti : [관광지명]

# 유사도 백분위수 상위 25% 이상인 관광지명 추출
for mbti in mbti_df['mbti']:
    tour_df_mbti[mbti] = tour_df_emb.loc[tour_df_emb[mbti].apply(lambda x: x >= tour_df_emb[mbti].quantile(0.75,interpolation='nearest'))].sort_values(ascending=False, by = mbti)['관광지명'].tolist()

### 유사도 백분위수 상위 25%인 장소명과 기존 데이터 병합

In [93]:
import os
for mbti in mbti_df['mbti']:
    for month in range(1,13):
        df = tour_dfs[f"tour_{month}"][tour_dfs[f"tour_{month}"]['관광지명'].apply(lambda x: x in tour_df_mbti[mbti])]
        if not os.path.exists(f'./data/tour_df/{mbti}'):
            os.mkdir(f'./data/tour_df/{mbti}')
        df.to_csv(f'./data/tour_df/{mbti}/tour_{month}_{mbti}.csv', index=False)

### 음식점과 MBTI 유형 간 유사도 계산

In [4]:
mct_df_emb = pd.DataFrame()

In [21]:
# 각 MBTI 유형에 대해 llm 0.5 other 0.5 유사도 계산
num = 0
for place, place_embedding in tqdm(review[['MCT_NM','llm5_other5']].values, desc='place'):
    for mbti, mbti_embedding in mbti_df[['mbti','embedding']].values:
    
        similarity = calculate_similarity(place_embedding, mbti_embedding)
        #print(f"{mbti} 유형과 {name} 키워드의 유사도: {similarity:.6f}")
        mct_df_emb.loc[num, '음식점명'] = place
        mct_df_emb.loc[num, mbti] = similarity
    num += 1

place: 100%|██████████| 5744/5744 [00:28<00:00, 200.08it/s]


### 유사도 백분위수 상위 25%인 장소명과 기존 데이터 병합

- 병하가 준 월별 mct 데이터

In [38]:
mct_dfs = {}

from glob import glob
csvs = glob('./data/mct_df/original/*.csv')
csvs.sort()
for i, j in enumerate(csvs):
    mct_dfs[f"mct_{i+1}"] = pd.read_csv(j)


In [39]:
mct_df_mbti = {} # mbti : [관광지명]

# 유사도 백분위수 상위 25% 이상인 관광지명 추출
for mbti in mbti_df['mbti']:
    mct_df_mbti[mbti] = mct_df_emb.loc[mct_df_emb[mbti].apply(lambda x: x >= mct_df_emb[mbti].quantile(0.75,interpolation='nearest'))].sort_values(ascending=False, by = mbti)['음식점명'].tolist()

In [3]:
# import os
# for mbti in mbti_df['mbti']:
#     for month in range(1,13):
#         df = mct_dfs[f"mct_{month}"][mct_dfs[f"mct_{month}"]['가맹점명'].apply(lambda x: x in mct_df_mbti[mbti])]
#         if not os.path.exists(f'./data/mct_df/{mbti}'):
#             os.mkdir(f'./data/mct_df/{mbti}')
                
#         # df['건당평균이용금액구간'] =  df['건당평균이용금액구간'].apply(lambda x: "상위 "+ x[2:] if not x[2:].startswith("상위") else x[2:])
#         # df['이용금액구간'] =  df['이용금액구간'].apply(lambda x: "상위 "+ x[2:] if not x[2:].startswith("상위") else x[2:])
#         # df['이용건수구간'] =  df['이용건수구간'].apply(lambda x: "상위 "+ x[2:] if not x[2:].startswith("상위") else x[2:])
#         df.to_csv(f'./data/mct_df/{mbti}/mct_{month}_{mbti}.csv', index=False)



In [33]:
import os
for mbti in mbti_df['mbti']:
    for month in range(1,13):
        df = mct_dfs[f"mct_{month}"][mct_dfs[f"mct_{month}"]['가맹점명'].apply(lambda x: x in mct_df_mbti[mbti])]
        if not os.path.exists(f'./data/mct_df/{mbti}'):
            os.mkdir(f'./data/mct_df/{mbti}')
        df.to_csv(f'./data/mct_df/{mbti}/mct_{month}_{mbti}.csv', index=False)

### 지오코딩 위도 경도 데이터와 리뷰 데이터 재방문율, 키워드 데이터 병합

In [122]:
geo_tour = pd.read_csv('jeju/geocoding/관광지_위도경도SUM.csv')
geo_mct = pd.read_csv('jeju/geocoding/JEJU_MCT_위도경도SUM.csv')

In [123]:
geo_mct = geo_mct[['MCT_NM','위도','경도']]
geo_mct.rename(columns={'MCT_NM':'가맹점명'}, inplace=True)
geo_mct.drop_duplicates(subset=['가맹점명'], keep='first', inplace=True)
geo_tour = geo_tour[['AREA_NM','위도','경도']]
geo_tour.rename(columns={'AREA_NM':'관광지명'}, inplace=True)
geo_tour.drop_duplicates(subset=['관광지명'], keep='first', inplace=True)

In [124]:
review_vm = review[['MCT_NM','visit_mean','LLM_keywords']]

In [125]:
review_vm.rename(columns={'MCT_NM':'가맹점명','visit_mean':'재방문율 평균','LLM_keywords':"리뷰키워드_20개"}, inplace=True)

/var/folders/y3/l8bvkgx53cd0nbrk2b450t7m0000gn/T/ipykernel_8254/2697788816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_vm.rename(columns={'MCT_NM':'가맹점명','visit_mean':'재방문율 평균','LLM_keywords':"리뷰키워드_20개"}, inplace=True)


In [126]:
review_vm.drop_duplicates(subset=['가맹점명'], keep='first', inplace=True)

/var/folders/y3/l8bvkgx53cd0nbrk2b450t7m0000gn/T/ipykernel_8254/1006817617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_vm.drop_duplicates(subset=['가맹점명'], keep='first', inplace=True)


,가맹점명,재방문율 평균,리뷰키워드_20개
0,올레길설렁탕,1.297872,"[설렁탕, 뚝불, 콩국수, 맛집, 도가니탕, 전골, 맛있다, 친절하다, 진하다, 시..."
1,돈도칸연동흑돼지,1.080000,"[흑돼지, 맛있다, 고기, 굽다, 직원, 친절하다, 소스, 톳, 땅콩, 빵가루, 육..."
2,돈사돈 한림협재점,1.060000,"돈사돈 : [흑돼지, 맛집, 맛있다, 친절하다, 최고, 굽다, 육즙, 부드럽다, 두..."
3,표선장수고을,2.157895,"[삼계탕, 맛집, 솥밥, 맛있다, 친절하다, 착하다, 푸짐하다, 든든하다, 정성, ..."
4,폴 바셋 제주이호DT점,1.400000,"폴바: 넓다, 쾌적하다, 조용하다, 맛있다, 친절하다, 뷰, 넓은, 깔끔하다, 좋다..."
...,...,...,...
5739,섬마당갈비,1.500000,"섬마당갈비 : 생갈비, 양념갈비, 맛있다, 푸짐하다, 가성비, 밑반찬, 계란찜, 양..."
5740,제주검정보리마씸,1.200000,"제주도 맛집 키워드:맛: - 맛있다, - 건강하다, - 시원하다, - 감칠맛,..."
5741,표선수산마트,1.080000,"[신선, 맛있다, 싱싱, 저렴, 가성비, 친절, 푸짐, 넉넉, 양, 많다, 착하다,..."
5742,할매추어탕,1.620000,"[추어탕, 게장, 맛있다, 든든하다, 건강하다, 깔끔하다, 친절하다, 푸짐하다, 신..."


In [127]:
review_vm['재방문율 평균'] = review_vm['재방문율 평균'].apply(lambda x: round(x,2))

/var/folders/y3/l8bvkgx53cd0nbrk2b450t7m0000gn/T/ipykernel_8254/3880458904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_vm['재방문율 평균'] = review_vm['재방문율 평균'].apply(lambda x: round(x,2))


In [56]:
mct_folders = os.listdir('./data/mct_df')
mct_folders.remove('.DS_Store')

In [70]:
mct_folders = [os.path.join('./data/mct_df',x) for x in mct_folders]
mct_folders_all = []
for f in mct_folders:
    files = glob(f+'/*.csv')
    for file in files:
        mct_folders_all.append(file)

In [67]:
tour_folders = os.listdir('./data/tour_df')
tour_folders.remove('.DS_Store')

In [68]:
tour_folders = [os.path.join('./data/tour_df',x) for x in tour_folders]
tour_folders_all = []
for f in tour_folders:
    files = glob(f+'/*.csv')
    for file in files:
        tour_folders_all.append(file)

- 관광지 데이터와 위도경도 데이터 병함

In [134]:
for file in tour_folders_all:
    df = pd.read_csv(file)
    df = pd.merge(df, geo_tour, on='관광지명', how='left')
    df.rename(columns={'text':"정보"}, inplace=True)
    df.to_csv(file, index=False)
    

- 음식점 데이터와 위도경도 데이터 병합, llm리뷰키워드, 재방문율 평균 추가

In [114]:
for file in mct_folders_all:
    df = pd.read_csv(file)
    df = pd.merge(df, geo_mct, on='가맹점명', how='left')
    df = pd.merge(df, review_vm, on='가맹점명', how='left')
    df.rename(columns={'text':"정보"},inplace=True)
    df['정보'] = df['정보']+" 해당 가맹점의 고객 평균 재방문율은 "+df['재방문율 평균'].astype(str) +" 입니다."+" 해당 가맹점에 대한 이용객의 리뷰 키워드는 다음과 같습니다: " + df['리뷰키워드_20개']
    df.to_csv(file, index=False)

- 장소명 중복값 제거

In [130]:
for file in mct_folders_all:
    df = pd.read_csv(file)
    df.drop_duplicates(subset='가맹점명', keep='first', inplace=True)
    df.to_csv(file, index=False)

for file in tour_folders_all:
    df = pd.read_csv(file)
    df.drop_duplicates(subset='관광지명', keep='first', inplace=True)
    df.to_csv(file, index=False)

### 기준월과 데이터명이 다른 경우 데이터 명 변경
### 중복 월은 없음

- tour_df는 민성이 위도 경도 코드 받아서 추가

In [1]:
import pandas as pd
import os
from glob import glob

In [2]:
mct_db_folders = os.listdir('./jeju/chatbot_arch/database/mct_db_havetofix')
#mct_db_folders.remove('.DS_Store')
mct_db_folders = [os.path.join('./jeju/chatbot_arch/database/mct_db_havetofix',x) for x in mct_db_folders]
mct_db_folders_all = []
for f in mct_db_folders:
    files = glob(f+'/*.faiss')
    files += glob(f+'/*.pkl')
    for file in files:
        mct_db_folders_all.append(file)
tour_db_folders = os.listdir('./jeju/chatbot_arch/database/tour_db_havetofix')
#tour_db_folders.remove('.DS_Store')
tour_db_folders = [os.path.join('./jeju/chatbot_arch/database/tour_db_havetofix',x) for x in tour_db_folders]
tour_db_folders_all = []
for f in tour_db_folders:
    files = glob(f+'/*.faiss')
    files += glob(f+'/*.pkl')
    for file in files:
        tour_db_folders_all.append(file)

In [19]:
mct_folders = os.listdir('./rename_data/mct_df_havetofix')
#mct_folders.remove('.DS_Store')
mct_folders = [os.path.join('./rename_data/mct_df_havetofix',x) for x in mct_folders]
mct_folders_all = []
for f in mct_folders:
    files = glob(f+'/*.csv')
    for file in files:
        mct_folders_all.append(file)
tour_folders = os.listdir('./rename_data/tour_df_havetofix')
#tour_folders.remove('.DS_Store')
tour_folders = [os.path.join('./rename_data/tour_df_havetofix',x) for x in tour_folders]
tour_folders_all = []
for f in tour_folders:
    files = glob(f+'/*.csv')
    for file in files:
        tour_folders_all.append(file)

In [4]:
havetofix = []
for file in mct_folders_all:
    df = pd.read_csv(file)
    rename = ""
    file_month = file.split('/')[-2:][-1].split('_')[1]
    data_month = str(df['기준연월'][0])[-2:]
    if data_month.startswith('0'):
        data_month = data_month[1:]
    if  file_month != data_month : # 파일명과 데이터 기준월이 다를 경우 파일명 변경
        rename = file.replace(file_month, data_month).replace('_havetofix','')
    else:
        rename = file.replace('_havetofix','')

    havetofix.append({'file':file, 'rename':rename})

    # if not os.path.exists(os.path.dirname(rename)):
    #     os.mkdir(os.path.dirname(rename))
    # df.to_csv(rename, index=False)
        #print(os.path.dirname(rename))
    

In [20]:
havetofix_tour = []
for file in tour_folders_all:
    df = pd.read_csv(file)
    rename = ""
    file_month = file.split('/')[-2:][-1].split('_')[1]
    data_month = str(int(df['기준월'][0]))
    #print(file_month, data_month)

    if data_month.startswith('0'):
        data_month = data_month[1:]
    if  file_month != data_month : # 파일명과 데이터 기준월이 다를 경우 파일명 변경
        rename = file.replace(file_month, data_month).replace('_havetofix','')
        #print(rename)
    else:
        rename = file.replace('_havetofix','')

    havetofix_tour.append({'file':file, 'rename':rename})

    # if not os.path.exists(os.path.dirname(rename)):
    #     os.mkdir(os.path.dirname(rename))
    # df.to_csv(rename, index=False)
    #     #print(os.path.dirname(rename))
    

In [29]:
for dic in havetofix_tour:
    df = pd.read_csv(dic['file'])
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0','Unnamed: 0.1'], inplace=True)
    df.to_csv(dic['rename'], index=False)

In [24]:
for dic in havetofix:
    df = pd.read_csv(dic['file'])
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)
    df.to_csv(dic['rename'], index=False)